## Import

In [1]:
import sys
sys.path.append(r'\\Pund\Stab$\guest801981\Documents\GitHub\GeoSpatial-analysis\facility-location-Bergen\src\facility_location_Bergen\custome_modules')
sys.path.append(r'C:\Users\Marco\Documents\GitHub\GeoSpatial-analysis\facility-location-Bergen\src\facility_location_Bergen\custome_modules')

In [2]:
import warnings
from shapely.errors import ShapelyDeprecationWarning
# Ignore the ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

In [3]:
import os
import dill
import pyproj
import folium
import requests
import numpy as np
import pandas as pd
import pickle as pkl
import networkx as nx
from scipy import stats
import plotly.graph_objects as go
from sklearn.utils import resample
# import cartopy.io.img_tiles as cimgt
from plotly.subplots import make_subplots
from urllib.request import urlopen, Request
from log import print_INFO_message_timestamp, print_INFO_message
from facility_location import AdjacencyMatrix, FacilityLocation, FacilityLocationReport

[    0.00] Initializing mpi-sppy


## Solution analysis

### Loading data

#### Exact solution

In [6]:
times = ["all_day", "all_day_free_flow", "morning", "midday", "afternoon"]
facilities_number = 2

In [48]:
print_INFO_message_timestamp("Loading exact solutions...")

fls_exact = {}

for time in times:
    print_INFO_message(f"Loading exact solution for {time}")
    path = rf"\\Pund\Stab$\guest801981\Documents\GitHub\GeoSpatial-analysis\facility-location-Bergen\data\07_model_output\{facilities_number}_locations\deterministic_exact_solutions\light_exact_solution_{time}.pkl"
    fls_exact[time] = FacilityLocation.load(path)

[09/13/23 00:06:18] INFO     Loading exact solutions...
                    INFO     Loading exact solution for all_day
                    INFO     Loading exact solution for all_day_free_flow
                    INFO     Loading exact solution for morning
                    INFO     Loading exact solution for midday
                    INFO     Loading exact solution for afternoon


In [843]:
# with open("prova.txt", "w") as f:
#     fls_exact["all_day"].instance.pprint(ostream=f)

#### dfs for solution comparison

In [7]:
root = rf"\\Pund\Stab$\guest801981\Documents\GitHub\GeoSpatial-analysis\facility-location-Bergen\data\08_reporting\{facilities_number}_locations"
paths = [p for p in os.listdir(root) if ("solution_vs_scenario" in p) and ("worst" not in p)]
paths_worst = [p for p in os.listdir(root) if ("solution_vs_scenario" in p) and ("worst" in p)]

In [8]:
dfs = {}

for path in paths:
    with open(os.path.join(root, path), "rb") as f:
        dfs[tuple(path.
                        replace("all_day_free_flow", "all-day-free-flow").
                        replace("all_day", "all-day").
                        removesuffix(".pkl").split("_")[-3:])] = pkl.load(f)

In [9]:
dfs.keys()

dict_keys([('afternoon', 'afternoon', 'weight'), ('afternoon', 'all-day', 'weight'), ('afternoon', 'all-day', 'weight2'), ('afternoon', 'midday', 'weight'), ('afternoon', 'morning', 'weight'), ('all-day', 'afternoon', 'weight'), ('all-day', 'all-day', 'weight'), ('all-day', 'all-day', 'weight2'), ('all-day-free-flow', 'afternoon', 'weight'), ('all-day-free-flow', 'all-day', 'weight'), ('all-day-free-flow', 'all-day', 'weight2'), ('all-day-free-flow', 'midday', 'weight'), ('all-day-free-flow', 'morning', 'weight'), ('all-day', 'midday', 'weight'), ('all-day', 'morning', 'weight'), ('midday', 'afternoon', 'weight'), ('midday', 'all-day', 'weight'), ('midday', 'all-day', 'weight2'), ('midday', 'midday', 'weight'), ('midday', 'morning', 'weight'), ('morning', 'afternoon', 'weight'), ('morning', 'all-day', 'weight'), ('morning', 'all-day', 'weight2'), ('morning', 'midday', 'weight'), ('morning', 'morning', 'weight')])

In [23]:
for t1, t2 in zip(dfs[('all-day-free-flow', 'all-day', 'weight2')].sort_values("source").travel_time,
          dfs[('all-day-free-flow', 'afternoon', 'weight')].sort_values("source").travel_time):
    
    if t1>t2:
        print(t1, t2)

11.425 11.421
11.56 11.559
11.13 11.126
11.248 11.226
11.056 11.05
12.253 11.792
10.421 10.401
10.196 10.17
14.464 13.038
10.026 9.997
9.615 9.577
16.048 13.931
16.608 14.247
11.109 11.104
16.693 14.295
16.693 14.295
15.266 13.49
14.066 12.814
11.404 11.401
9.05 9.045
9.074 9.065
9.087 9.076
9.087 9.076
9.499 9.459
11.056 11.05
11.148 11.144
9.457 9.415
34.093 34.07
34.093 34.07


In [866]:
dfs_worst = {}

for path in paths_worst:
    with open(os.path.join(root, path), "rb") as f:
        print()
        dfs_worst[tuple(path.
                        replace("all_day_free_flow", "all-day-free-flow").
                        replace("all_day", "all-day").
                        removesuffix(".pkl").split("_")[-4:-1])] = pkl.load(f)

#### Adj matrix mapping

In [5]:
adj_mappings = {}

for time in times:
    if time != "all_day_free_flow":
        print_INFO_message(f"Loading adj mapping for {time}")
        path = rf"C:\Users\Marco\Documents\GitHub\GeoSpatial-analysis\facility-location-Bergen\data\03_primary\adj_mapping_{time}.pkl"
        with open(path, "rb") as f:
            adj_mappings[time] = dill.load(f)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 adj_mappings = {}                                                                            │
│   2                                                                                              │
│ ❱ 3 for time in times:                                                                           │
│   4 │   if time != "all_day_free_flow":                                                          │
│   5 │   │   print_INFO_message(f"Loading adj mapping for {time}")                                │
│   6 │   │   path = rf"C:\Users\Marco\Documents\GitHub\GeoSpatial-analysis\facility-location-     │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'times' is not defined

#### Average graph

In [33]:
average_graphs = {}

for time in times:
    if time != "all_day_free_flow":
        print_INFO_message(f"average graphs for {time}")
        path = rf"\\Pund\Stab$\guest801981\Documents\GitHub\GeoSpatial-analysis\facility-location-Bergen\data\03_primary\average_graph_{time}.pkl"
        with open(path, "rb") as f:
            average_graphs[time] = pkl.load(f)

                    INFO     average graphs for all_day


                    INFO     average graphs for morning
                    INFO     average graphs for midday
                    INFO     average graphs for afternoon


In [39]:
c_min = 0
c_gre = 0
for e in average_graphs["midday"].edges(data=True):
    if e[2]["weight"] < e[2]["weight2"]:
        c_min += 1
    else:       
        c_gre += 1
        
print(c_min, c_gre)

960 8996


In [7]:
worst_average_graphs = {}

for time in times:
    if time != "all_day_free_flow":
        print_INFO_message(f"worst average graphs for {time}")
        path = rf"C:\Users\Marco\Documents\GitHub\GeoSpatial-analysis\facility-location-Bergen\data\03_primary\worst_average_graph_{time}.pkl"
        with open(path, "rb") as f:
            worst_average_graphs[time] = pkl.load(f)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 worst_average_graphs = {}                                                                    │
│   2                                                                                              │
│ ❱ 3 for time in times:                                                                           │
│   4 │   if time != "all_day_free_flow":                                                          │
│   5 │   │   print_INFO_message(f"worst average graphs for {time}")                               │
│   6 │   │   path = rf"C:\Users\Marco\Documents\GitHub\GeoSpatial-analysis\facility-location-     │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'times' is not defined

### Exact solution analysis

In [14]:
print_INFO_message_timestamp("Objective value for the Exact solution")
for time, fl_exact in fls_exact.items():
    print_INFO_message(f"{time}: {round(fl_exact.solution_value/60,3)} minutes")

[06/21/23 09:33:18] INFO     Objective value for the Exact solution
                    INFO     all_day: 19.334 minutes


In [ ]:
report_exact = FacilityLocationReport(fls_exact)

In [ ]:
report_exact.graphical_keys_solutions_comparison()

- Let O denote the objective function, t denote time, and x denotes the decision variables. 
- Let x_ff be the value of the decision variables obtained in the free-flow setting. 

For each time condition t, we want to compute the value of the objective function O(x_ff, t) when fixing x to be x_ff.

### Retrieve average graphs

In [ ]:
fls_exact["all_day_free_flow"].locations_coordinates

In [ ]:
sv = fls_exact["all_day_free_flow"].solution_value/60
sv

In [ ]:
a = fls_exact["all_day_free_flow"].adjacency_matrix
print(a.shape)
np.where(a/60 == sv, a, 0).nonzero()

In [ ]:
print(f'source: {adj_mappings["all_day"][1442]}\ndestination: {adj_mappings["all_day"][646]}')

In [ ]:
a = nx.dijkstra_path_length(G=average_graphs["all_day"], 
                            source=adj_mappings["all_day"][1442], 
                            target=adj_mappings["all_day"][646],
                            weight="weight2")/60

b = fls_exact["all_day_free_flow"].adjacency_matrix[1442, 646]/60

print(f"shortest path lenght: {a}\nadj matrix: {b}")

### Compare solution under different scenarios

In [12]:
def solution_vs_scenario(time_solution, time_scenario, weight="weight2", worst=False):
    
    # Load the exact solution
    print_INFO_message_timestamp(f"Loading exact solution for {time_solution}")
    path = rf"C:\Users\Marco\Documents\GitHub\GeoSpatial-analysis\facility-location-Bergen\data\07_model_output\{facilities_number}_locations\deterministic_exact_solutions\exact_solution_{time_solution}.pkl"
    fls_exact_solution = FacilityLocation.load(path)
    
    # Load the average graph
    print_INFO_message(f"Loading adj matrix for {time_scenario}")
    if worst:
        path = rf"C:\Users\Marco\Documents\GitHub\GeoSpatial-analysis\facility-location-Bergen\data\03_primary\worst_average_graph_{time_scenario}.pkl"
    else:
        path = rf"C:\Users\Marco\Documents\GitHub\GeoSpatial-analysis\facility-location-Bergen\data\03_primary\average_graph_{time_scenario}.pkl"
    
    with open(path, "rb") as f:
        average_graph = pkl.load(f)
    
    # extract the coordinates of the exact solution
    ff_solutions_location = fls_exact_solution.locations_coordinates
    
    # compute the distance from the exact solution to all the other nodes in the graph
    print_INFO_message_timestamp(f"Compute the distance from the {time_solution} solution to all the other nodes in the {time_scenario} graph")
    
    temporal_distances = {ff_solutions_location[i].geometry.coords[0]: [] for i in range(len(ff_solutions_location))}
    coordinates = [(x,y) for x,y in zip(fls_exact_solution.coordinates.geometry.x, fls_exact_solution.coordinates.geometry.y)]
    
    for i, node in enumerate(average_graph):
        if i%500 == 0:
            print_INFO_message(f"{i} out of {len(average_graph.nodes)}")
        
        if node in coordinates:    
            keys = list(temporal_distances.keys())
            temporal_distances[keys[0]].append(
                (node, nx.dijkstra_path_length(G=average_graph, 
                                    source=keys[0], 
                                    target=node,
                                    weight=weight))
                ) 
        else:
            continue
        # temporal_distances[keys[1]].append(
        #     (node, nx.dijkstra_path_length(G=average_graph, 
        #                         source=keys[1], 
        #                         target=node,
        #                         weight=weight))
        #     )
    
    # create a dataframe with the distance from the exact solution to all the other nodes in the graph
    d = {"source": [], "target": [], "travel_time": []}
    
    for key, value in temporal_distances.items():
        for node, distance in value:
            d["source"].append(key)
            d["target"].append(node)
            d["travel_time"].append(round(distance/60, 3))
            
    return pd.DataFrame(d)

In [29]:
def get_minimum_distances(df):
    return df.groupby("target").\
                                apply(lambda x: x.sort_values(by="travel_time", ascending=True).iloc[0]).\
                                drop("target", axis=1).\
                                reset_index()

#### Free-flow

In [868]:
dfs.keys()

dict_keys([('afternoon', 'afternoon', 'weight'), ('afternoon', 'all-day', 'weight'), ('afternoon', 'all-day', 'weight2'), ('afternoon', 'midday', 'weight'), ('afternoon', 'morning', 'weight'), ('all-day', 'afternoon', 'weight'), ('all-day', 'all-day', 'weight'), ('all-day', 'all-day', 'weight2'), ('all-day-free-flow', 'afternoon', 'weight'), ('all-day-free-flow', 'all-day', 'weight'), ('all-day-free-flow', 'all-day', 'weight2'), ('all-day-free-flow', 'midday', 'weight'), ('all-day-free-flow', 'morning', 'weight'), ('all-day', 'midday', 'weight'), ('all-day', 'morning', 'weight'), ('midday', 'afternoon', 'weight'), ('midday', 'all-day', 'weight'), ('midday', 'all-day', 'weight2'), ('midday', 'midday', 'weight'), ('midday', 'morning', 'weight'), ('morning', 'afternoon', 'weight'), ('morning', 'all-day', 'weight'), ('morning', 'all-day', 'weight2'), ('morning', 'midday', 'weight'), ('morning', 'morning', 'weight')])

In [869]:
df_all_day_ff_min = get_minimum_distances(dfs[('all-day-free-flow', 'all-day', 'weight2')])

In [870]:
df_all_day_ff_min.sort_values(by="travel_time", ascending=False).reset_index().head(5)

,index,target,source,travel_time
0,554,"(5.31708, 60.3945)","(5.33809, 60.34675)",20.456
1,538,"(5.31497, 60.39507)","(5.33809, 60.34675)",20.149
2,4,"(5.19042, 60.34358)","(5.33809, 60.34675)",19.788
3,862,"(5.34806, 60.26287)","(5.33809, 60.34675)",19.669
4,865,"(5.34876, 60.26311)","(5.33809, 60.34675)",19.593


In [871]:
df_all_day_ff_min["travel_time"].mean()

10.853283035714286

In [873]:
a = round(fls_exact["all_day_free_flow"].solution_value/60, 3)
b = dfs[('all-day-free-flow', 'all-day', 'weight2')].groupby("target").min().sort_values(by="travel_time", ascending=False).iloc[0].travel_time

print(f"exact solution: {a}\nff approximation: {b}\nrel_difference: {round(abs(a-b)/a * 100,3)}")

exact solution: 20.456
ff approximation: 20.456
rel_difference: 0.0


#### All-day

In [874]:
df_all_day_min = get_minimum_distances(dfs[("all-day-free-flow", "all-day", "weight")])
df_worst_all_day_min = get_minimum_distances(dfs_worst[("all-day-free-flow", "all-day", "weight")])

In [875]:
print(f'average time: {df_all_day_min["travel_time"].mean()}\naverage worst time: {df_worst_all_day_min["travel_time"].mean()}')

average time: 12.005969642857144
average worst time: 13.320900892857143


In [876]:
a = round(fls_exact["all_day"].solution_value/60, 3)

b = df_all_day_min.sort_values(by="travel_time", ascending=False).iloc[0].travel_time
b_worst = df_worst_all_day_min.sort_values(by="travel_time", ascending=False).iloc[0].travel_time

rel_difference_all_day = round(abs(a-b)/a * 100,3)
rel_difference_all_day_worst = round(abs(a-b_worst)/a * 100,3)

print(f"exact solution: {a}\nff approximation: {b}\nrel_difference: {rel_difference_all_day}\nrel_difference_worst: {rel_difference_all_day_worst}")

exact solution: 22.002
ff approximation: 25.497
rel_difference: 15.885
rel_difference_worst: 28.875


#### Morning

In [877]:
df_morning_min = get_minimum_distances(dfs[("all-day-free-flow", "morning", "weight")])
df_worst_morning_min = get_minimum_distances(dfs_worst[("all-day-free-flow", "morning", "weight")])

In [878]:
print(f'average time: {df_morning_min["travel_time"].mean()}\naverage worst time: {df_worst_morning_min["travel_time"].mean()}')

average time: 11.910085714285714
average worst time: 13.125370535714287


In [879]:
a = round(fls_exact["morning"].solution_value/60, 3)

b = df_morning_min.sort_values(by="travel_time", ascending=False).iloc[0].travel_time
b_worst = df_worst_morning_min.sort_values(by="travel_time", ascending=False).iloc[0].travel_time

rel_difference_morning = round(abs(a-b)/a * 100,3)
rel_difference_morning_worst = round(abs(a-b_worst)/a * 100,3)

print(f"exact solution: {a}\nff approximation: {b}\nrel_difference: {rel_difference_morning}\nrel_difference_worst: {rel_difference_morning_worst}")

exact solution: 21.886
ff approximation: 25.382
rel_difference: 15.974
rel_difference_worst: 33.432


#### Midday

In [880]:
df_midday_min = get_minimum_distances(dfs[("all-day-free-flow", "midday", "weight")])
df_worst_midday_min = get_minimum_distances(dfs_worst[("all-day-free-flow", "midday", "weight")])

In [881]:
print(f'average time: {df_midday_min["travel_time"].mean()}\naverage worst time: {df_worst_midday_min["travel_time"].mean()}')

average time: 11.718003571428572
average worst time: 12.770939285714286


In [882]:
a = round(fls_exact["midday"].solution_value/60, 3)

b = df_midday_min.sort_values(by="travel_time", ascending=False).iloc[0].travel_time
b_worst = df_worst_midday_min.sort_values(by="travel_time", ascending=False).iloc[0].travel_time

rel_difference_midday = round(abs(a-b)/a * 100,3)
rel_difference_midday_worst = round(abs(a-b_worst)/a * 100,3)

print(f"exact solution: {a}\nff approximation: {b}\nrel_difference: {rel_difference_midday}\nrel_difference_worst: {rel_difference_midday_worst}")

exact solution: 22.404
ff approximation: 25.596
rel_difference: 14.247
rel_difference_worst: 26.781


#### Afternoon

In [884]:
df_afternoon_min = get_minimum_distances(dfs[("all-day-free-flow", "afternoon", "weight")])
df_worst_afternoon_min = get_minimum_distances(dfs_worst[("all-day-free-flow", "afternoon", "weight")])

In [885]:
print(f'average time: {df_afternoon_min["travel_time"].mean()}\naverage worst time: {df_worst_afternoon_min["travel_time"].mean()}')

average time: 12.271916071428572
average worst time: 13.92019107142857


In [886]:
a = round(fls_exact["afternoon"].solution_value/60, 3)

b = df_afternoon_min.sort_values(by="travel_time", ascending=False).iloc[0].travel_time
b_worst = df_worst_afternoon_min.sort_values(by="travel_time", ascending=False).iloc[0].travel_time

rel_difference_afternoon = round(abs(a-b)/a * 100,3)
rel_difference_afternoon_worst = round(abs(a-b_worst)/a * 100,3)

print(f"exact solution: {a}\nff approximation: {b}\nrel_difference: {rel_difference_afternoon}\nrel_difference_worst: {rel_difference_afternoon_worst}")

exact solution: 21.965
ff approximation: 25.674
rel_difference: 16.886
rel_difference_worst: 30.986


### Reporting

In this section, the following steps are performed:
1. **Compare the objective function value** under different scenarios, for a specif set of solution locations
   
2. Given the matrix containing the travel times between all OD pairs, for a specific solution, **compare the distribution** of travel times between OD pairs under different scenarios
   
3. **Dispaly the path** associated to the solutions at step1 

#### Step 1: Compare the objective function value under different scenarios, for a specif set of solution locations

In [887]:
rel_diffs = [rel_difference_all_day, rel_difference_morning, rel_difference_midday, rel_difference_afternoon]
rel_diffs_worst = [rel_difference_all_day_worst, rel_difference_morning_worst, rel_difference_midday_worst, rel_difference_afternoon_worst]

In [888]:
fig = make_subplots(rows=1, cols=1,)
fig.update_layout(title="<b>Outsample evaluation, relative differences<b>",
                  title_pad_l=75,
                  height=500,
                  width=600,
                  yaxis_title="relative difference [%]")

fig.update_yaxes(range=[0, 100])

fig.add_trace(go.Bar(y=rel_diffs, 
                     name="average scenario",
                     marker=dict(color=["blue"]*len(rel_diffs)),
                     x=["all_day", "morning", "midday", "afternoon"],), row=1, col=1)

fig.add_trace(go.Bar(y=rel_diffs_worst,
                     name="average worst scenario",
                     marker=dict(color=["navy"]*len(rel_diffs_worst)),
                     x=["all_day", "morning", "midday", "afternoon"],), row=1, col=1)

fig.update_layout(legend=dict(
                            orientation='h',  # Set the orientation to 'h' for horizontal
                            yanchor='bottom',  # Anchor the legend to the bottom
                            y=1.02,  # Adjust the y position to place the legend below the figure
                            xanchor='left',  # Anchor the legend to the left
                            x=0  # Adjust the x position if necessary
                        ),)

#### Step 2: Compare the distribution of travel times between OD pairs under different scenarios

In [889]:
df_min = df_all_day_ff_min[["target", "travel_time"]]

for df, name in zip([df_all_day_min, df_morning_min, df_midday_min, df_afternoon_min, 
                     df_worst_all_day_min, df_worst_morning_min, df_worst_midday_min, df_worst_afternoon_min], 
                    ["all_day", "morning", "midday", "afternoon", "worst_all_day", "worst_morning", "worst_midday", "worst_afternoon"]):
    
    df_min = df_min.merge(df[["target", "travel_time"]], 
                          on="target", 
                          suffixes=(None, "_"+name),
                          how="outer")

df_min = df_min.rename(columns={"travel_time": "travel_time_free_flow"})

In [890]:
fig = go.Figure()

show_legend = [True]+[False]*len(df_min.columns[1:])

fig.update_layout(title="<b>Distribution for free flow travel times solution across average scenarios<b>",
                  title_pad_l=150,
                  height=500,
                  width=1200,
                  xaxis_title="time of the day",)

fig.update_yaxes(range=[0, 40])

for i, name in enumerate(["free_flow", "all_day", "morning", "midday", "afternoon",
                          "worst_all_day", "worst_morning", "worst_midday", "worst_afternoon"]):
    fig.add_trace(go.Violin(y=df_min["travel_time_free_flow"],
                            name=name,
                            box_visible=True,
                            meanline_visible=False,
                            hoverinfo="none",
                            side="negative",
                            line_color="lightseagreen",
                            showlegend=show_legend[i]))
    
    fig.add_trace(go.Violin(y=df_min["travel_time_"+name],
                            name=name,
                            box_visible=True,
                            meanline_visible=False,
                            hoverinfo="none",
                            side="positive",
                            line_color="mediumpurple",
                            showlegend=show_legend[-1]))

fig.show()

In [891]:
# Perform Mann-Whitney U test
for col in df_min.columns[2:]:
    print_INFO_message_timestamp(f"Performing Mann-Whitney U test for {col}")
    statistic, p_value = stats.mannwhitneyu(df_min["travel_time_free_flow"], df_min[col])

    # Print the results
    print_INFO_message(f"Mann-Whitney U statistic: {statistic}")
    print_INFO_message(f"P-value: {p_value}")
    print("\n")

[07/06/23 14:23:44] INFO     Performing Mann-Whitney U test for travel_time_all_day
                    INFO     Mann-Whitney U statistic: 533199.0
                    INFO     P-value: 8.169196544538626e-10


[07/06/23 14:23:44] INFO     Performing Mann-Whitney U test for travel_time_morning
                    INFO     Mann-Whitney U statistic: 540185.5
                    INFO     P-value: 1.3070893655018727e-08


[07/06/23 14:23:44] INFO     Performing Mann-Whitney U test for travel_time_midday
                    INFO     Mann-Whitney U statistic: 555091.5
                    INFO     P-value: 2.4621516846183497e-06


[07/06/23 14:23:44] INFO     Performing Mann-Whitney U test for travel_time_afternoon
                    INFO     Mann-Whitney U statistic: 518289.5
                    INFO     P-value: 1.1132217781144972e-12


[07/06/23 14:23:44] INFO     Performing Mann-Whitney U test for travel_time_worst_all_day
                    INFO     Mann-Whitney U statistic: 445370.0
  

In [892]:
mean_ci = pd.DataFrame({"mean": None, "lower_bound": None, "upper_bound": None}, 
                       index=df_min.columns[1:])

for col in df_min.columns[1:]:
    # Number of bootstrap iterations
    n_iterations = 1000

    # Confidence level (e.g., 95%)
    confidence_level = 0.95

    # Array to store bootstrap sample statistics
    bootstrap_means = []

    # Perform bootstrap iterations
    for _ in range(n_iterations):
        bootstrap_sample = resample(df_min[col], replace=True, n_samples=len(df_min))
        bootstrap_mean = np.mean(bootstrap_sample)
        bootstrap_means.append(bootstrap_mean)

    # Compute confidence interval
    lower_bound = np.percentile(bootstrap_means, (1 - confidence_level) / 2 * 100)
    upper_bound = np.percentile(bootstrap_means, (1 + confidence_level) / 2 * 100)

    # Add to dataframe
    mean_ci.loc[col] = [df_min[col].mean(), lower_bound, upper_bound]
    
# Print the confidence interval
mean_ci = mean_ci.sort_values(by="mean", ascending=False).round(3)

In [893]:
mean_ci

,mean,lower_bound,upper_bound
travel_time_worst_afternoon,13.920191,13.59982,14.280769
travel_time_worst_all_day,13.320901,13.01035,13.61933
travel_time_worst_morning,13.125371,12.816754,13.436807
travel_time_worst_midday,12.770939,12.48377,13.049085
travel_time_afternoon,12.271916,11.992371,12.547469
travel_time_all_day,12.00597,11.733446,12.267284
travel_time_morning,11.910086,11.649597,12.188684
travel_time_midday,11.718004,11.460789,11.967706
travel_time_free_flow,10.853283,10.614892,11.075524


In [894]:
fig = go.Figure()

fig.update_layout(title="<b>Average travel time for free flow solution across average scenarios<b>",
                  title_pad_l=130,
                  height=600,
                  width=1100,
                  xaxis_title="time of the day",
                  yaxis_title="mean travel time [min]")

fig.add_trace(go.Bar(x=mean_ci.index, 
                     y = mean_ci["mean"],
                     width=0.5,
                     name='mean'))

# Add the vertical line
for col in df_min.columns[1:]:
        fig.add_shape(type='line',
                x0=col, y0=mean_ci.loc[col]["lower_bound"],
                x1=col, y1=mean_ci.loc[col]["upper_bound"],
                xref='x', yref='y',
                line=dict(color='red', width=10))

fig.update_yaxes(range=[0, mean_ci["upper_bound"].max()+1])

fig.show()

#### Step 3: Dispaly the path associated to the solutions at step1

In [56]:
root = rf"\\Pund\Stab$\guest801981\Documents\GitHub\GeoSpatial-analysis\facility-location-Bergen\data\08_reporting\{facilities_number}_locations"
paths = [p for p in os.listdir(root) if ("solution_vs_scenario" in p) and ("worst" not in p)]
# paths_worst = [p for p in os.listdir(root) if ("solution_vs_scenario" in p) and ("worst" in p)]

In [57]:
dfs_min = {}

for path in paths:
    with open(os.path.join(root, path), "rb") as f:
        df = pkl.load(f)
        df = get_minimum_distances(df).sort_values(by="travel_time", ascending=False)
        dfs_min[tuple(path.
                      replace("all_day_free_flow", "all-day-free-flow").
                      replace("all_day", "all-day").
                      removesuffix(".pkl").split("_")[-3:])] = df

In [58]:
dfs_min.keys()

dict_keys([('afternoon', 'afternoon', 'weight'), ('afternoon', 'all-day', 'weight'), ('afternoon', 'all-day', 'weight2'), ('afternoon', 'midday', 'weight'), ('afternoon', 'morning', 'weight'), ('all-day', 'afternoon', 'weight'), ('all-day', 'all-day', 'weight'), ('all-day', 'all-day', 'weight2'), ('all-day-free-flow', 'afternoon', 'weight'), ('all-day-free-flow', 'all-day', 'weight'), ('all-day-free-flow', 'all-day', 'weight2'), ('all-day-free-flow', 'midday', 'weight'), ('all-day-free-flow', 'morning', 'weight'), ('all-day', 'midday', 'weight'), ('all-day', 'morning', 'weight'), ('midday', 'afternoon', 'weight'), ('midday', 'all-day', 'weight'), ('midday', 'all-day', 'weight2'), ('midday', 'midday', 'weight'), ('midday', 'morning', 'weight'), ('morning', 'afternoon', 'weight'), ('morning', 'all-day', 'weight'), ('morning', 'all-day', 'weight2'), ('morning', 'midday', 'weight'), ('morning', 'morning', 'weight')])

In [59]:
dfs_min[('afternoon', 'afternoon', 'weight')]

,target,source,travel_time
960,"(5.46883, 60.41773)","(5.33642, 60.37613)",55.553
921,"(5.45076, 60.37664)","(5.33642, 60.37613)",54.238
980,"(5.50474, 60.42368)","(5.33642, 60.37613)",53.655
981,"(5.51179, 60.42282)","(5.33642, 60.37613)",53.302
982,"(5.513479, 60.422659)","(5.33642, 60.37613)",53.217
...,...,...,...
408,"(5.302465, 60.384249)","(5.33642, 60.37613)",4.407
483,"(5.3163, 60.38215)","(5.33642, 60.37613)",2.782
484,"(5.31636, 60.38213)","(5.33642, 60.37613)",2.773
610,"(5.3311, 60.37709)","(5.33642, 60.37613)",0.672


In [959]:
sources = {}
destinations = {}
solution_paths = {}

for key, df in dfs_min.items():
    try:
        if (key[0] == key[1] and key[2] == "weight") or (key[0] == "all-day-free-flow" and 
                                key[1] == "all-day" and key[2] == "weight2"):
            sources[key] = df.iloc[0]["source"]
            destinations[key] = df.iloc[0]["target"]
            solution_paths[key] = nx.dijkstra_path(G=average_graphs[key[1].replace("-", "_")],
                                                source=sources[key],
                                                target=destinations[key],
                                                weight=key[2])
            # print(key[1].replace("-", "_"))
            # print(nx.dijkstra_path_length(G=average_graphs[key[1].replace("-", "_")],
            #                                     source=sources[key],
            #                                     target=destinations[key],
            #                                     weight=key[2])/60)
        else:
            continue
    except:
        print("Skipping: ", key[0])

In [961]:
def get_travel_time(solution_path, graph, weight):
    travel_time = 0
    for i in range(len(solution_path)-1):
        sp = solution_path[i]
        ep = solution_path[i+1]
        travel_time += graph.get_edge_data(sp, ep)[weight]
    return travel_time

In [996]:
travel_time = {}
for key in solution_paths.keys():
    travel_time[key] = {}
    for time in ['all_day_free_flow', 'all_day', 'morning', 'midday', 'afternoon']:
        if key[0].replace("-", "_") == time:
            if time == "all_day_free_flow":
                travel_time[key][time] = nx.dijkstra_path_length(G=average_graphs["all_day"], source=sources[key], target=destinations[key], weight=key[2])
            else:
                # print(key[0].replace("-", "_"), time)
                travel_time[key][time] = nx.dijkstra_path_length(G=average_graphs[time], source=sources[key], target=destinations[key], weight=key[2])
        else:
            if time == "all_day_free_flow":
                try:
                    travel_time[key][time] = get_travel_time(solution_paths[key], average_graphs["all_day"], "weight2")
                except:
                    print(key, solution_paths[key])
            else:
                travel_time[key][time] = get_travel_time(solution_paths[key], average_graphs[time], "weight")
        try:
            minutes = int(travel_time[key][time]/60)
            seconds = int(travel_time[key][time]%60)
            travel_time[key][time] = str(minutes) + " min" + " " + str(seconds) + " sec"
        except:
            continue

In [42]:
# for time in ['all_day', 'morning', 'midday', 'afternoon']:
#     travel_time[time+"_worst"] = get_travel_time(solution_path, worst_average_graphs[time])
#     minutes = int(travel_time[time+"_worst"]/60)
#     seconds = int(travel_time[time+"_worst"]%60)
#     travel_time[time+"_worst"] = str(minutes) + " min" + " " + str(seconds) + " sec"

In [43]:
# for time in travel_time.keys():
#     print(f"travel time {time}: {travel_time[time]}")

travel time free_flow: 27 min 35 sec
travel time all_day: 29 min 53 sec
travel time morning: 29 min 23 sec
travel time midday: 29 min 55 sec
travel time afternoon: 30 min 2 sec
travel time all_day_worst: 32 min 9 sec
travel time morning_worst: 30 min 25 sec
travel time midday_worst: 31 min 8 sec
travel time afternoon_worst: 33 min 32 sec


##### Folium Map

In [997]:
center_pt = [60.41, 5.32415]
color_mapping = {
    "all-day-free-flow":"red",
    "all-day":"black",
    "morning":"blue",
    "midday":"orange",
    "afternoon":"green",
}
map = folium.Map(location=center_pt, tiles="OpenStreetMap", zoom_start=9.5)

tooltip_targets = {}
for key in travel_time.keys():
    target = solution_paths[key][-1]
    if target not in tooltip_targets.keys():
        tooltip_targets[target] = f"<b>Farthest location for:</b>" 
    
    tooltip_targets[target] += f"<br>- {key[0].upper()}"

for key in travel_time.keys():
    tooltip_source =f"<b>{key[0].upper()}</b><br>(opt locations)<br><br>"+"<br>- ".join(["<b>Travel time</b>:"]+[rf"{time}: " + travel_time[key][time] for time in 
                               ['all_day_free_flow', 'all_day', 'morning', 'midday', 'afternoon']])

    start_marker = folium.Marker(location=(solution_paths[key][0][1]+np.random.normal(0, 0.0003, 1),
                                           solution_paths[key][0][0]+np.random.normal(0, 0.0003, 1)),
                icon=folium.Icon(color=color_mapping[key[0]], prefix='fa',icon='car'),
                tooltip=tooltip_source)
    start_marker.add_to(map)
    
    folium.Marker(location=(solution_paths[key][-1][1], solution_paths[key][-1][0]),
                  tooltip=tooltip_targets[solution_paths[key][-1]],
                  icon=folium.Icon(color='gray', prefix='fa',icon='crosshairs'),).add_to(map)

    path = folium.PolyLine(locations=[(node[1], node[0]) for node in solution_paths[key]], 
                    color=color_mapping[key[0]],
                    tooltip=key[0],
                    weight=2,)

    path.add_to(map)
    

map

## Choerency analysis

### Part 1

In [7]:
from retrieve_global_parameters import *

In [61]:
time = "morning"
facilities_number = 1
handpicked=True

In [62]:
root = rf"\/Pund/Stab$/guest801981/Documents/GitHub/GeoSpatial-analysis/facility-location-Bergen/"

In [63]:
print_INFO_message_timestamp("Loading exact solutions...")

fls_exact = {}

for time in [time]:
    print_INFO_message(f"Loading exact solution for {time}")
    path = root + retrieve_light_solution_path(facilities_number,time,handpicked)
    fls_exact[time] = FacilityLocation.load(path)

[09/21/23 15:07:52] INFO     Loading exact solutions...
                    INFO     Loading exact solution for morning


In [64]:
fls_exact[time].solution_value/60

0.0

In [65]:
# adj_matrix = fls_exact[time].adjacency_matrix/60
with open(root+retrieve_adj_matrix_path(time, False, handpicked), "rb") as f:
     adj_matrix = pkl.load(f)/60
     
with open(root+retrieve_adj_mapping_path(time, False, handpicked), "rb") as f:
     adj_mapping = pkl.load(f)
     
with open(root+retrieve_adj_mapping_path_2(time, False, handpicked), "rb") as f:
     adj_mapping_2 = pkl.load(f)

In [66]:
adj_mapping_reverse = {v:k for k,v in adj_mapping.items()}

In [67]:
idx1 = fls_exact[time].candidate_coordinates.index
idx2 = fls_exact[time].coordinates.index

In [72]:
fls_exact[time].candidate_coordinates

,geometry
7220,POINT (5.19351 60.36165)
3156,POINT (5.19581 60.36734)
3210,POINT (5.21048 60.37185)
7294,POINT (5.23121 60.37170)
7346,POINT (5.23355 60.39204)
3063,POINT (5.24348 60.36128)
3464,POINT (5.24978 60.36833)
355,POINT (5.25315 60.28372)
369,POINT (5.25325 60.29001)
8707,POINT (5.25641 60.26679)


In [69]:
adj_matrix = adj_matrix[idx1, :][:, idx2]

In [70]:
np.max(adj_matrix, axis=1).min()

27.73925570103389

In [24]:
i_max = np.argmax(adj_matrix, axis=1)
i_min = np.max(adj_matrix, axis=1).argmin()

In [25]:
i_max[i_min], i_min

(386, 18)

In [26]:
tg = fls_exact[time].coordinates.iloc[i_max[i_min]][0].coords[0]
tg_index = adj_mapping_reverse[tg]

sr = fls_exact[time].candidate_coordinates.iloc[i_min][0].coords[0]
sr_index = adj_mapping_reverse[sr]

print(f"source: {sr}\nsource index: {sr_index}\ntarget: {tg}\ntarget index: {tg_index}")

[09/21/23 15:04:16] WARNING  C:\Users\guest801981\AppData\Local\Temp\ipykernel_14448\3258182034.py: warnings.py:109
                             1: FutureWarning: Series.__getitem__ treating keys as positions is                    
                             deprecated. In a future version, integer keys will always be treated                  
                             as labels (consistent with DataFrame behavior). To access a value by                  
                             position, use `ser.iloc[pos]`                                                         
                               tg = fls_exact[time].coordinates.iloc[i_max[i_min]][0].coords[0]                    
                                                                                                                   

                    WARNING  C:\Users\guest801981\AppData\Local\Temp\ipykernel_14448\3258182034.py: warnings.py:109
                             4: FutureWarning: Series.__getitem__ treating keys as positions is                    
                             deprecated. In a future version, integer keys will always be treated                  
                             as labels (consistent with DataFrame behavior). To access a value by                  
                             position, use `ser.iloc[pos]`                                                         
                               sr = fls_exact[time].candidate_coordinates.iloc[i_min][0].coords[0]                 
                                                                                                                   

source: (5.340075, 60.3768)
source index: 8415
target: (5.48472, 60.43864)
target index: 77


In [27]:
sr_index_mapped, tg_index_mapped = adj_mapping_2[(sr_index, tg_index)]
sr_mapped = adj_mapping[sr_index_mapped]
tg_mapped = adj_mapping[tg_index_mapped]

print(f"source: {sr_mapped}\ntarget: {tg_mapped}")

source: (5.34007, 60.37687)
target: (5.48472, 60.43864)


In [28]:
fls_exact[time].solution_value/60

28.474206296002617

In [29]:
if time == "all_day_free_flow":
    print(nx.dijkstra_path_length(G=average_graphs["all_day"], source=sr_mapped, target=tg_mapped, weight="weight2")/60)
else:
    print(nx.dijkstra_path_length(G=average_graphs[time], source=sr_mapped, target=tg_mapped, weight="weight")/60)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│   1 if time == "all_day_free_flow":                                                              │
│   2 │   print(nx.dijkstra_path_length(G=average_graphs["all_day"], source=sr_mapped, target=     │
│   3 else:                                                                                        │
│ ❱ 4 │   print(nx.dijkstra_path_length(G=average_graphs[time], source=sr_mapped, target=tg_ma     │
│   5                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'average_graphs' is not defined

In [30]:
fls_exact[time].locations_coordinates, fls_exact[time].locations_index


([geometry    POINT (5.34007 60.37680)
Name: 8415, dtype: geometry], [8415])

In [31]:
fls_exact[time].locations_coordinates[0].name

8415

In [24]:
if time == "all_day_free_flow":
    path = nx.dijkstra_path(G=average_graphs["all_day"], source=sr, target=tg, weight="weight2")
else:
    path = nx.dijkstra_path(G=average_graphs[time], source=sr, target=tg, weight="weight")

### Part 2

In [48]:
times = ["all_day_free_flow", "all_day", "morning", "midday", "afternoon"]
fls_exact = {}
print_INFO_message_timestamp("Loading exact solutions...")
for time in times:
    print_INFO_message(f"Loading exact solution for {time}")
    path = rf"\\Pund\Stab$\guest801968\Documents\GitHub\GeoSpatial-analysis\facility-location-Bergen\data\07_model_output\{facilities_number}_locations\deterministic_exact_solutions\light_exact_solution_{time}.pkl"
    fls_exact[time] = FacilityLocation.load(path)

[09/06/23 09:13:24] INFO     Loading exact solutions...
                    INFO     Loading exact solution for all_day_free_flow
                    INFO     Loading exact solution for all_day
                    INFO     Loading exact solution for morning
                    INFO     Loading exact solution for midday
                    INFO     Loading exact solution for afternoon


In [49]:
idx1 = {}
idx2 = {}
for time in times:
    idx1[time] = fls_exact[time].candidate_coordinates.index
    idx2[time] = fls_exact[time].coordinates.index

In [50]:
adj_matrix = {}
for time in times:
    with open(rf"\\Pund\Stab$\guest801968\Documents\GitHub\GeoSpatial-analysis\facility-location-Bergen\data\03_primary\adj_matrix_{time}.pkl", "rb") as f:
        adj_matrix[time] = pkl.load(f)

In [51]:
for time in times:
    adj_matrix[time] = adj_matrix[time]/60
    adj_matrix[time] = adj_matrix[time][idx1[time], :][:, idx2[time]]

In [52]:
for time in times:
    print(f"{time}: {np.max(adj_matrix[time], axis=1).min()}")

all_day_free_flow: 33.37759475154043
all_day: 35.6050654286313
morning: 34.803502062052004
midday: 35.29619602517862
afternoon: 36.496001389117616


In [53]:
for time in times:
    i_max = np.argmax(adj_matrix[time], axis=1)
    i_min = np.max(adj_matrix[time], axis=1).argmin()
    print(f"\ntime: {time}")
    print(f"target: {fls_exact[time].coordinates.iloc[i_max[i_min]]}\nsource: {fls_exact[time].candidate_coordinates.iloc[i_min]}")


time: all_day_free_flow
target: geometry    POINT (5.44920 60.44748)
Name: 5356, dtype: geometry
source: geometry    POINT (5.36390 60.30903)
Name: 1019, dtype: geometry

time: all_day
target: geometry    POINT (5.46883 60.41773)
Name: 121, dtype: geometry
source: geometry    POINT (5.34014 60.37445)
Name: 1627, dtype: geometry

time: morning
target: geometry    POINT (5.46883 60.41773)
Name: 121, dtype: geometry
source: geometry    POINT (5.34014 60.37445)
Name: 1626, dtype: geometry

time: midday
target: geometry    POINT (5.44920 60.44748)
Name: 5357, dtype: geometry
source: geometry    POINT (5.36390 60.30903)
Name: 950, dtype: geometry

time: afternoon
target: geometry    POINT (5.26593 60.50495)
Name: 9530, dtype: geometry
source: geometry    POINT (5.32774 60.29848)
Name: 2326, dtype: geometry
